In [3]:
!pip install farm-haystack
from haystack.document_store.elasticsearch import ElasticsearchDocumentStore

In [4]:
doc_store = ElasticsearchDocumentStore(
    host='localhost',
    username='', password='',
    index='essays'
)

05/31/2021 19:26:36 - INFO - elasticsearch -   HEAD http://localhost:9200/ [status:200 request:21.573s]
05/31/2021 19:26:37 - INFO - elasticsearch -   HEAD http://localhost:9200/essays [status:200 request:1.081s]
05/31/2021 19:26:37 - INFO - elasticsearch -   GET http://localhost:9200/essays [status:200 request:0.123s]
05/31/2021 19:26:41 - INFO - elasticsearch -   PUT http://localhost:9200/essays/_mapping [status:200 request:3.717s]
05/31/2021 19:26:41 - INFO - elasticsearch -   HEAD http://localhost:9200/label [status:200 request:0.046s]


In [5]:
from haystack.retriever.dense import DensePassageRetriever

In [6]:
retriever = DensePassageRetriever(
    document_store=doc_store,
    query_embedding_model='facebook/dpr-question_encoder-single-nq-base',
    passage_embedding_model='facebook/dpr-ctx_encoder-single-nq-base',
    use_gpu=True,
    embed_title=True
)

In [6]:
doc_store.update_embeddings(retriever=retriever)

05/28/2021 20:54:14 - INFO - elasticsearch -   POST http://localhost:9200/essays/_count [status:200 request:0.972s]
05/28/2021 20:54:14 - INFO - haystack.document_store.elasticsearch -   Updating embeddings for all 17208 docs ...
05/28/2021 20:54:16 - INFO - elasticsearch -   POST http://localhost:9200/essays/_search?scroll=1d&size=10000 [status:200 request:1.703s]
Creating Embeddings: 100%|███████████████████████████████████████████████████| 625/625 [1:31:43<00:00,  8.81s/ Batches]
05/28/2021 22:29:13 - INFO - elasticsearch -   POST http://localhost:9200/_bulk?refresh=wait_for [status:200 request:47.336s]
05/28/2021 22:29:19 - INFO - elasticsearch -   POST http://localhost:9200/_bulk?refresh=wait_for [status:200 request:5.227s]
05/28/2021 22:29:31 - INFO - elasticsearch -   POST http://localhost:9200/_bulk?refresh=wait_for [status:200 request:12.177s]
05/28/2021 22:29:39 - INFO - elasticsearch -   POST http://localhost:9200/_bulk?refresh=wait_for [status:200 request:7.688s]
05/28/2021

In [7]:
retriever.retrieve('what do we need to replicate the experiment')

05/31/2021 19:27:36 - INFO - elasticsearch -   POST http://localhost:9200/essays/_search [status:200 request:32.478s]


[{'text': "In order to replicate this experiment, you would need to know what the samples were. In this group's procedure, they do not state what", 'id': '0b152a38-7f69-4a0e-abbf-bd1df0c84a6a', 'score': 73.35169999999994, 'probability': 0.6755765790370192, 'question': None, 'meta': {'source': 'essays'}, 'embedding': None},
 {'text': 'To replicate this experiment one must know information that is not given by the procedure. First, we must know how much vinegar to put into the containers, enough to cover them? Next, we need to know how the vinegar will affect the material. Will  we completely submerge it or only partially? Finally, we need to know which materials to use. The data indicates them, but the procedure does not.', 'id': '93ddd43e-6f51-4b7b-8bae-49f6660bb35b', 'score': 71.62349999999992, 'probability': 0.6717774002399869, 'question': None, 'meta': {'source': 'essays'}, 'embedding': None},
 {'text': 'To exactly replicate this experiment you would have to know, how they measured 

In [9]:
from haystack.reader.farm import FARMReader

In [18]:
reader = FARMReader(model_name_or_path="deepset/roberta-base-squad2",
                use_gpu=True, no_ans_boost=-10, context_window_size=500,
                top_k_per_candidate=3, top_k_per_sample=1,
                num_processes=8, max_seq_len=256, doc_stride=128)

05/31/2021 19:44:15 - INFO - farm.utils -   Using device: CPU 
05/31/2021 19:44:15 - INFO - farm.utils -   Number of GPUs: 0
05/31/2021 19:44:15 - INFO - farm.utils -   Distributed Training: False
05/31/2021 19:44:15 - INFO - farm.utils -   Automatic Mixed Precision: None
Some weights of RobertaModel were not initialized from the model checkpoint at deepset/roberta-base-squad2 and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
05/31/2021 19:44:38 - WARNING - farm.utils -   ML Logging is turned off. No parameters, metrics or artifacts will be logged to MLFlow.
05/31/2021 19:44:38 - INFO - farm.utils -   Using device: CPU 
05/31/2021 19:44:38 - INFO - farm.utils -   Number of GPUs: 0
05/31/2021 19:44:38 - INFO - farm.utils -   Distributed Training: False
05/31/2021 19:44:38 - INFO - farm.utils -   Automatic Mixed Precision: None
05/31/2021 19

In [19]:
pipeline = ExtractiveQAPipeline(reader, retriever)

In [20]:
from transformers import AutoModelForQuestionAnswering,AutoTokenizer,pipeline

In [21]:
from haystack.pipeline import ExtractiveQAPipeline
pipeline = ExtractiveQAPipeline(reader, retriever)

In [22]:
# You can configure how many candidates the reader and retriever shall return
# The higher top_k_retriever, the better (but also the slower) your answers. 
prediction = pipeline.run(query="what do we need to replicate the experiment?", top_k_retriever=10, top_k_reader=5)

05/31/2021 19:45:17 - INFO - elasticsearch -   POST http://localhost:9200/essays/_search [status:200 request:8.394s]
Inferencing Samples: 100%|█████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  2.03 Batches/s]


In [23]:
from haystack.utils import print_answers

In [24]:
print_answers(prediction, details="minimal")

[   {   'answer': 'how they measured the mass of the object',
        'context': 'To exactly replicate this experiment you would have to '
                   'know, how they measured the mass of the object, because '
                   'without that how would we figure out the different masses. '
                   'We have to know the starting mass, the ending mass and the '
                   'difference in mass.'},
    {   'answer': 'the four different samples used, materials used in this '
                  'experiment and a start time and ending time',
        'context': '1To replicate this experiment you would need the four '
                   'different samples used, materials used in this experiment '
                   'and a start time and ending time.'},
    {   'answer': 'to know what the samples were',
        'context': 'In order to replicate this experiment, you would need to '
                   "know what the samples were. In this group's procedure, "
                

In [25]:
# Make sure you have a GPU running
!nvidia-smi

'nvidia-smi' is not recognized as an internal or external command,
operable program or batch file.
